In [33]:
from hw2skeleton import cluster as cl
from hw2skeleton import io
import os
import pandas as pd
import numpy as np
aa3 = "ALA CYS ASP GLU PHE GLY HIS ILE LYS LEU MET ASN PRO GLN ARG SER THR VAL TRP TYR".split()
aa_df = pd.DataFrame(0, index=list(aa3), columns=['Count'])

In [34]:
def compute_similarity_matrix(sites):
    simMat = []
    names = []
    for i in range(len(sites)):
        names.append(sites[i].name)
        row = []
        for j in range(len(sites)):
            row.append(cl.compute_similarity(sites[i].counts,sites[j].counts))
        simMat.append(row)
    simMat = pd.DataFrame(simMat, columns = names, index = names)
    
    return simMat

In [35]:
def find_min(simMat):
    min_value = simMat[simMat!=0].min()[0]
    min_pair = [simMat[simMat!=0].idxmin()[0], simMat.index[0]]
    for i in simMat:
        if simMat[simMat!=0][i].min() < min_value:
            min_pair = (simMat[simMat!=0][i].idxmin(), i)
            min_value = simMat[simMat!=0][i].min()
    return list(min_pair)

In [36]:
def rm_most_similar(df, pair):
    return df.drop(pair, axis=0).drop(pair, axis =1)
def name_cluster(num):
    return str('c' + str(num))

In [37]:
def compute_cluster_center(cluster_list, sites_dict, aa_df):
    sites = aa_df
    for j in cluster_list:
        sites += sites_dict[j].counts 
    return sites / len(sites)

In [38]:
sites = io.read_active_sites('data')
sites_dict = {}
for site in sites:
    sites_dict[site.name] = site

Read in 136 active sites


In [55]:
simMat = compute_similarity_matrix(sites)

In [40]:
def compute_new_cluster_sim(new_clust_avg, simMat_update, sites_dict, clusters):
    newSim = []
    for site in simMat_update.columns:
        if site not in sites_dict:
            s = compute_cluster_center(clusters[site], sites_dict, aa_df)
        else:
            s = sites_dict[site].counts
        
        newSim.append(cl.compute_similarity(new_clust_avg, s))
    newSim.append(0.0)
    return newSim
    

In [41]:
def update_simMat(newSim, simMat_update, new_name):
    simMat_update[new_name] = None
    newRow = pd.DataFrame([newSim], columns = simMat_update.columns, index = [new_name])
    simMat_update = simMat_update.append(newRow)
    simMat_update[new_name] = newSim
    
    return simMat_update

In [52]:
def update_cluster_dict(new_name, min_pair, clusters, sites_dict):
    pair0 = []
    pair1 = []
    if min_pair[0] in clusters:
        pair0 += unpack_cluster(clusters[min_pair[0]], sites_dict, clusters)
        del clusters[min_pair[0]]
    else:
        pair0.append(sites_dict[min_pair[0]].name)
    if min_pair[1] in clusters:
        pair1 += unpack_cluster(clusters[min_pair[1]], sites_dict, clusters)
        del clusters[min_pair[1]]
    else:
        pair1.append(sites_dict[min_pair[1]].name)
    clusters[new_name] = pair0 + pair1
    
    return clusters
    

In [43]:
def unpack_loop(clusters, key, out_list):
    for value in clusters[key]:
        out_list.append(value)
    return out_list
    

In [44]:
def unpack_cluster(cluster_list, sites_dict, clusters):
    out_list = []
    for key in cluster_list:
        if key not in sites_dict:
            for value in clusters[key]:
                out_list.append(value)
        else:
            out_list.append(key)
    return out_list

try to unpack combo clusters everytime they get inserted into a new cluster so that deep recursion isn't necessary

In [56]:
simMat_update = simMat.copy()
c = 0
clusters = {}
while len(simMat_update) > 1:
    print(c)
    new_name = name_cluster(c)
    c += 1
    min_pair=find_min(simMat_update)
    clust_sites = unpack_cluster(min_pair, sites_dict, clusters)
    simMat_update = rm_most_similar(simMat_update, min_pair)
    new_clust_avg = compute_cluster_center(clust_sites, sites_dict, aa_df)
    newSim = compute_new_cluster_sim(new_clust_avg, simMat_update, sites_dict, clusters)
    simMat_update = update_simMat(newSim, simMat_update, new_name)
    clusters = update_cluster_dict(new_name, min_pair, clusters, sites_dict)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
